In [1]:
import papermill as pm
import scrapbook as sb
import pandas as pd
from datetime import datetime, timedelta
import os

C:\Users\15103\Conda\envs\lecture_4\lib\site-packages\papermill\iorw.py:50: FutureWarning: pyarrow.HadoopFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  from pyarrow import HadoopFileSystem


In [2]:
prices = pd.read_sql(f'''
SELECT 
    * 
FROM ohlc
''', 'sqlite:///data/data.db')
prices['ts'] = pd.to_datetime(prices['ts'])

prices.set_index('ts', inplace=True)

In [3]:
prices.token.unique()

array(['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV',
       'AAVE', 'COMP'], dtype=object)

In [4]:
tokens=['BTC', 'ETH', 'USDT', 'SOL', 'ADA', 'DOT', 'AVAX', 'ATOM', 'CRV', 'AAVE', 'COMP']

In [5]:
prices.loc[prices.token=='BTC'].sort_index(ascending=False)

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-15 00:00:00,48288.99,48398.68,48164.49,48219.09,106.582858,5.145144e+06,BTC,BTC
2021-12-14 23:00:00,48303.56,48482.90,48008.28,48288.99,675.097958,3.259071e+07,BTC,BTC
2021-12-14 22:00:00,47805.73,48686.91,47773.01,48303.57,1804.082095,8.714276e+07,BTC,BTC
2021-12-14 21:00:00,46875.95,47890.62,46856.37,47805.73,1634.610294,7.759534e+07,BTC,BTC
2021-12-14 20:00:00,46658.23,47086.98,46649.03,46877.70,783.748050,3.677161e+07,BTC,BTC
...,...,...,...,...,...,...,...,...
2021-11-01 04:00:00,60590.23,60655.00,59752.92,59971.89,621.419878,3.744744e+07,BTC,BTC
2021-11-01 03:00:00,61333.17,61457.28,60050.00,60589.06,1059.931358,6.414625e+07,BTC,BTC
2021-11-01 02:00:00,61610.94,61779.87,61299.89,61333.17,314.250720,1.935390e+07,BTC,BTC


In [6]:
prices.loc[prices.token=='COMP'].sort_index(ascending=False)

,open,high,low,close,volume,volumeUSD,token,chain
ts,,,,,,,,
2021-12-15 00:00:00,185.07,185.78,184.50,184.89,335.307,6.214264e+04,COMP,ETH
2021-12-14 23:00:00,185.22,186.09,183.88,185.08,416.028,7.698720e+04,COMP,ETH
2021-12-14 22:00:00,183.31,186.74,182.60,185.24,2086.017,3.865710e+05,COMP,ETH
2021-12-14 21:00:00,180.15,183.76,179.96,183.43,2453.455,4.447119e+05,COMP,ETH
2021-12-14 20:00:00,178.99,181.27,178.72,180.28,628.699,1.130815e+05,COMP,ETH
...,...,...,...,...,...,...,...,...
2021-11-01 04:00:00,336.20,337.04,327.40,327.75,2391.802,7.975887e+05,COMP,ETH
2021-11-01 03:00:00,343.79,344.64,332.70,335.81,3615.275,1.220542e+06,COMP,ETH
2021-11-01 02:00:00,345.72,350.47,343.00,343.46,2771.400,9.616592e+05,COMP,ETH


It looks like the time range for each token is the same, so I can set the from-date and to-date the same for each token

In [7]:
index = prices.loc[prices.token=='COMP'].sort_index(ascending=False).index

In [8]:
date = index.date
to_date = date[0]-timedelta(days= date[0].weekday()) #get the last Monday in the time series 
from_date = to_date - timedelta(days=14) #get the third last Monday in the time series
connection_string = 'sqlite:///data/data.db'

In [9]:
base_output_folder = os.path.expanduser('outputs')

if not os.path.exists(base_output_folder):
        os.makedirs(base_output_folder)

In [10]:
for token in tokens:
    print(f'running for {token} for date {from_date} to {to_date-timedelta(days=1)}', end='\r')
    res = pm.execute_notebook(
        'Daisy Dai HW4.ipynb',
        f'{base_output_folder}/market_analysis_{token}_{from_date}.ipynb',
        parameters = {
            'from_date': f'{from_date}',
            'to_date': f'{to_date}',
            'token': token,
            'connection_string': connection_string,
            }
    )

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

...

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

Executing:   0%|          | 0/19 [00:00<?, ?cell/s]

In [11]:
nbs = sb.read_notebooks('outputs')

In [12]:
pd.DataFrame([
    pd.Series({
        'token': scrap['token'].data,
        'from_date': scrap['from_date'].data,
        'to_date': scrap['to_date'].data,
        'average hourly return': scrap['prices'].data['hourly return'].mean(),
        'average volatility': scrap['prices'].data.volatility.mean(),
        'maximum drawdown':scrap['prices'].data['maximum drawdown'][-1],
        'range': scrap['prices'].data.close[-1] - scrap['prices'].data.close[0]}) 
    for book, scrap in nbs.notebook_scraps.items()
])

,token,from_date,to_date,average hourly return,average volatility,maximum drawdown,range
0,AAVE,2021-11-29,2021-12-13,-0.000806,0.230765,15.4870,-58.6450
1,ADA,2021-11-29,2021-12-13,-0.000421,0.241399,0.1204,-0.2445
2,ATOM,2021-11-29,2021-12-13,-0.000271,0.333404,2.8710,-2.7560
3,AVAX,2021-11-29,2021-12-13,-0.000609,0.319050,7.4100,-22.9300
4,BTC,2021-11-29,2021-12-13,-0.000331,0.166549,2762.2400,-6999.8400
5,COMP,2021-11-29,2021-12-13,-0.001057,0.242214,23.0400,-88.2800
6,CRV,2021-11-29,2021-12-13,-0.000712,0.364842,0.4181,-1.0362
7,DOT,2021-11-29,2021-12-13,-0.000453,0.271552,2.3840,-6.0050
8,ETH,2021-11-29,2021-12-13,-0.000048,0.192750,187.8700,-141.7100
9,SOL,2021-11-29,2021-12-13,-0.000361,0.270302,17.3720,-26.6870
